In [1]:
import pandas as pd
import os
from pprint import pprint
from api_key import *
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.agents.agent_types import AgentType
from langchain_openai import OpenAI

# Estudo de agentes 
O contúdo da explicação teóricas dos agentes está no obsias. Aqui é um estudo exploratório de algumas funcionalidades. Para isso vamos começar com um agente capaz de manipular dados em pandas para poder manipular uma base de dados do total de Bolos que podem ser vendidos no dia

In [16]:
df = pd.read_csv("dados.csv")
agent = create_pandas_dataframe_agent(OpenAI(temperature=0.1, model="gpt-3.5-turbo-0613"), df, verbose=True)
df

,Quantidade,Produtos
0,5,Bolo de Cenoura
1,3,Bolo de Chocolate
2,3,Bolo de Ninho com Nutella
3,2,Bolo de Crocante de amendoin
4,3,Bolo de Abacaxi
5,2,Bolo de Milho
6,1,Mini Bolo Churros
7,2,Mini Bolo de Cenoura
8,1,Mini Bolo de Ninho com Nutella
9,3,Mini Bolo Amenduim


Note que a o modelo se enrolou todo e ainda não cumpriu o requisitado. Talvez valha a pena desenvolver uma ferramenta para que seja possível ele fazer isso. Agora vamos tentar dar personalidade ao chatbot, para isso usamos os os promt templates de chats:

In [17]:
chat_template = ChatPromptTemplate.from_messages( 
		[ 
		("system", "Você é a assistente de uma doceira e boleira, e está ajudando ela a vender bolos."+ 
                    "Você atende cliente leigos que so querem informações sobre um produto, "+
                    "então use uma linguagem sem jargões e fale em português. Os bolos dispóniveis são os contidos "+
                    "no DataFrame. Seu nome é {name}."), 
		("human", "Oi, tudo bem?"), 
		("ai", "Estou muito bem,  o que você deseja para hoje?"), 
		("human", "{user_input}"),
		] ) 

formatted_messages = chat_template.format_messages(name="Bruxa", user_input="Quantos bolos de cenoura ainda tem?") 
for message in formatted_messages: 
	print(message)

content='Você é a assistente de uma doceira e boleira, e está ajudando ela a vender bolos.Você atende cliente leigos que so querem informações sobre um produto, então use uma linguagem sem jargões e fale em português. Os bolos dispóniveis são os contidos no DataFrame. Seu nome é Bruxa.'
content='Oi, tudo bem?'
content='Estou muito bem,  o que você deseja para hoje?'
content='Quantos bolos de cenoura ainda tem?'


In [18]:
response = agent.invoke(formatted_messages)



> Entering new AgentExecutor chain...


NotFoundError: Error code: 404 - {'error': {'message': 'This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?', 'type': 'invalid_request_error', 'param': 'model', 'code': None}}

Infelizmente o bot só está respondendo em inglês, para resolver isso criamos outro chatbot que irá taduzir o output do outro.

In [15]:
def tradutor(english_response):
    output = english_response['output']
    bot = OpenAI()
    return bot(f'Por favor traduza esse texo para português: {output}')
print(tradutor(response))

(content='Olá, tudo bem? Como posso ajudá-lo?')]

Este é o resultado de `print(df.tail())`:
|    |   Quantidade | Produtos                     |
|---:|-------------:|:-----------------------------|
|  5 |            3 | Bolo de Ninho com Nutella    |
|  6 |            2 | Bolo de Crocante de amendoin |
|  7 |            3 | Bolo de Abacaxi              |
|  8 |            2 | Bolo de Ninho com Nutella    |
|  9 |            3 | Bolo de Ninho com Nutella    |

Comece!
Pergunta: [MensagemDoSistema(conteúdo='Você é a assistente de uma doceira e boleira, e está ajudando ela a vender bolos. Você atende clientes leigos que só querem informações sobre um produto, então use uma linguagem sem jargões e fale em português. Os bolos disponíveis estão listados no DataFrame. Seu nome é Bruxa.'), MensagemHumana(conteúdo='Oi, tudo bem?'), Mensagem
